PRÁCTICA 1 - TIPOLOGÍA Y CICLO DE VIDA DE LOS DATOS

Caso práctico orientado a aprender a identificar los datos relevantes para un proyecto analítico y usar herramientas de extracción de datos. 

El objetivo de esta actividad será la creación de un dataset a partir de los datos contenidos en un sitio web. 

In [1]:
import requests
import csv

In [2]:
from bs4 import BeautifulSoup

In [3]:
# Función descargaPaginaWeb que permite recuperar la información correspondiente a la respuesta de la petición. 
# url: corresponde con la url del sitio donde se hará el web scraping
def descargaPaginaWeb(url):
    respuesta = requests.get(
        url,
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0'
        }
    )
    return respuesta

# Funcion extraerCaracteristica que actualiza el valor de la caracteristica según el valor de posicion. La función retorna
# un diccionario (sismo) al que se le ha añadido la característica
# caracteristica: característica
# posicion: indica qué etiqueta tiene la característica
# sismo: almacena todas las caracteristicas de un sismo en un diccionario
def extraerCaracteristica(caracteristica, posicion, sismo):
    if posicion == 0:
        sismo['Evento'] = caracteristica
    elif posicion == 1:
        sismo['Fecha'] = caracteristica
    elif posicion == 2:
        sismo['HoraUTC'] = caracteristica
    elif posicion == 3:
        sismo['HoraLocal'] = caracteristica
    elif posicion == 4:
        sismo['Latitud'] = caracteristica
    elif posicion == 5:
        sismo['Longitud'] = caracteristica
    elif posicion == 6:
        sismo['Profundidad'] = caracteristica
    elif posicion == 7:
        sismo['Magnitud'] = caracteristica
    elif posicion == 8:
        sismo['Tipo_Magnitud'] = caracteristica
    elif posicion == 9:
        sismo['Intensidad'] = caracteristica
    elif posicion == 10:
        sismo['Localizacion'] = caracteristica
    elif posicion == 11:
        sismo['Imagen'] = caracteristica
    return sismo

# Función extraerImagen que extrae la url de una imagen contenida en otra url de información sobre un sismo sobre la que
# hay que hacer web scraping. La función retorna la url de la imagen o NA si no hay información de la imagen.
# tag: etiqueta html
# parametro: parametro que se añade a la etiqueta
def extraerImagen(tag, parametro):
    info_form = tag.find('form')
    output = info_form['action'] + '?evid=' + parametro
    pagina = descargaPaginaWeb(output)
    contenido_web = BeautifulSoup(pagina.text, 'html.parser')
    imagen = contenido_web.find('img', attrs={"alt": "Foto Detalle"})
    if type(imagen['src']) != 'str':
        return 'NA'
    else:
        return imagen['src']
    
        
# Función extraerDatos que extrae de una página web la información requerida. La función retorna una lista con la 
# información de todos los sismos producidos (cada sismo se almacena en un diccionario con los nombres de los campos y 
# sus valores).
# respuesta: respuesta obtenida al preguntar a una url específica
def extraerDatos(respuesta):
    sismos = []
    contenido_web = BeautifulSoup(respuesta.text, 'html.parser')
    tabla_sismos_30 = contenido_web.table
    filas = tabla_sismos_30.findAll('td')
    elemento = 0
    sismo = {}
    for fila in filas:
        if elemento < 11:
            sismo = extraerCaracteristica(fila.text.strip(), elemento, sismo)
            elemento = elemento + 1
        else:
            # Si se quiere guardar el archivo gif con la imagen del lugar del sismo
            #sismo = extraerCaracteristica(extraerImagen(fila, sismo['Evento']), elemento, sismo)
            sismo = extraerCaracteristica('NA', elemento, sismo)
            elemento = 0
            sismos.append(sismo)
            sismo = {}
    return sismos

# Función crearArchivoCSVDesdeLista que crea un archivo csv a partir de una lista de diccionarios.
# lista: lista con los diccionarios
# nombreArchivo: nombre del archivo csv
def crearArchivoCSVDesdeLista(lista, nombreArchivo):
    campos = ['Evento', 'Fecha', 'HoraUTC', 'HoraLocal', 'Latitud', 'Longitud', 'Profundidad', 'Magnitud', 
              'Tipo_Magnitud', 'Intensidad', 'Localizacion', 'Imagen']
    try:
        with open(nombreArchivo + '.csv', 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=campos)
            writer.writeheader()
            for elemento in lista:
                writer.writerow(elemento)
    except IOError:
        print("I/O error")


# Función principal
def main():
    respuesta = descargaPaginaWeb('https://www.ign.es/web/ign/portal/ultimos-terremotos/-/ultimos-terremotos/get30dias')
    lista = extraerDatos(respuesta)
    crearArchivoCSVDesdeLista(lista, 'dataset')

In [4]:
main()